In [1]:
from google.colab import drive#On monte le drive pour charger les datasets depuis google drive (optionnel)
drive.mount('/content/gdrive/', force_remount=True)
%cd gdrive/MyDrive/

Mounted at /content/gdrive/
/content/gdrive/MyDrive


In [2]:
import pandas as pd
X = pd.read_csv("Melted_Train.csv")
y = pd.read_csv("Melted_Fused_Labels.csv")
dfXTest = pd.read_csv("Melted_Test.csv")


In [7]:
import xgboost as xgb
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.4,random_state=42)

dtrain = xgb.DMatrix(X_train,label=y_train)
dtest = xgb.DMatrix(X_test,label=y_test)

model = xgb.XGBModel()
params = {
    'max_depth': 10,
    'eta': 0.1,
    'objective': 'binary:logistic',
    'eval_metric': 'error',
    'booster':'dart',
    'sample_type':'weighted',
    'normalize_type':'forest',
    'learning_rate':0.4,
    'tree_method':'gpu_hist',
    'n_jobs':500,
    'rate_drop':0.3,
    'skip_drop':0.2,

}

# Train model
evalist = [(dtrain, 'train'), (dtest, 'eval')]
model = xgb.train(params, dtrain,num_boost_round=200,verbose_eval=1)

preds = model.predict(dtest) #Predictions
y_test = y_test.astype(float)
print("Precision = {}".format(average_precision_score(y_test.values, preds, average='macro')))
print("Precision = {}".format(average_precision_score(y_test.values, preds, average='weighted')))


Precision = 0.25674527434942446
Precision = 0.4107742871167801


In [8]:
dFin = xgb.DMatrix(dfXTest)
preds_f = model.predict(dFin)

dfF = pd.DataFrame(preds_f, columns=["Washing Machine","Dishwasher","Tumble Dryer","Microwave","Kettle"])
dfF = dfF.rename_axis('Index')
dfF.to_csv("no_features_baseline.csv")